In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
#from pmdarima import auto_arima
# Ignore SettingWithCopyWarning
warnings.filterwarnings("ignore")
# Assuming you have a hierarchical DataFrame named 'sales_data' with columns 'Foodcourt', 'Restaurant', 'Item'
df = pd.read_excel("meals_std.xlsx")
df = df.reset_index(drop = True)
df = df[["foodcourt","restaurant","restaurantmenuitem","date","total_count"]]
len(df)

112196

In [2]:
# pip install xgboost
# pip install openpyxl
# #pip install openpyxl
#python -m pip install prophet



In [3]:
# grouping the FRI (Food court- Restaurant- Item Combination) to create a input
def group_fri(df):
    grouped_df = df.groupby(['foodcourt', 'restaurant', 'restaurantmenuitem', 'date']).agg({'total_count': 'sum'}).reset_index()
    return grouped_df

# Example usage:
# Assuming df is your DataFrame
df = group_fri(df)
len(df)
#df.to_csv("group.csv")

103696

In [4]:
def df_summary(df, train_end, num_months):
    # Convert end_date to datetime object
    end_date = datetime.strptime(train_end, '%Y-%m-%d')
    
    # Calculate start date based on end date and number of months
    start_date = end_date - timedelta(days=30*num_months) + timedelta(days=1)  # Adjusting start_date based on num_months
    
    # Filter dataframe based on date range
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    filtered_df["period"] = num_months
    filtered_df['fri'] = filtered_df['foodcourt'] + '_' + filtered_df['restaurant'] + '_' + filtered_df['restaurantmenuitem']
    # Group by period and sum total counts
    grouped_df = filtered_df.groupby('period').agg({'total_count': 'sum','foodcourt': 'nunique','restaurant': 'nunique','restaurantmenuitem': 'nunique','fri': 'nunique'}).reset_index()
    grouped_df = grouped_df[["period","foodcourt","restaurant","restaurantmenuitem","fri","total_count"]]
    return grouped_df

## Output

train_end = '2024-02-11'
num_months =  6 # Change this to 1, 2, or 6 as needed
#summary_df = df_summary(df, train_end, num_months)

summary_df_1 = df_summary(df, train_end, 1)
summary_df_2 = df_summary(df, train_end, 2)
summary_df_6 = df_summary(df, train_end, 6)
summary_df =  pd.concat([summary_df_1,summary_df_2,summary_df_6])
summary_df

,period,foodcourt,restaurant,restaurantmenuitem,fri,total_count
0,1,101,273,1860,1952,333501
0,2,102,294,2203,2348,627637
0,6,102,384,3432,3610,1731391


In [5]:
def rank_dataframe2(df, train_date, num_months):
    # Convert date to datetime object
    end_date = datetime.strptime(train_date, '%Y-%m-%d')
    
    # Calculate start date based on end date and number of months
    start_date = end_date - timedelta(days=30*num_months) + timedelta(days=1)  # Adjusting start_date based on num_months
    
    # Filter dataframe based on date range
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= date)]
    grouped_df = filtered_df.groupby(['foodcourt', 'restaurant', 'restaurantmenuitem']).agg({'total_count': 'mean'}).reset_index()
    # Rename 'total_count' column to 'average_count'
    grouped_df.rename(columns={'total_count': 'average_sales'}, inplace=True)
    # Sort by average sales and assign ranks
    grouped_df['rank'] = grouped_df['average_sales'].rank(ascending=False).astype(int)
        
    # Create sales_segment based on total_count ranges
    def sales_segment(row):
        count = row['average_sales']
        if count > 100:
            return '>100'
        elif 75 <= count <= 100:
            return '75-100'
        elif 50 <= count < 75:
            return '50-75'
        elif 25 <= count < 50:
            return '25-50'
        elif 10 <= count < 25:
            return '10-25'
        else:
            return '<10'
    # Create rank_segment based on rank ranges
    def rank_segment(row):
        count = row['rank']
        if count <= 100:
            return '1-100'
        elif 100 < count <= 200:
            return '100-200'
        elif 200 < count <= 300:
            return '200-300'
        elif 300 < count <= 400:
            return '300-400'
        elif 400 < count <= 500:
            return '400-500'
        else:
            return '>500'
    
    grouped_df['sales_segment'] = grouped_df.apply(sales_segment, axis=1)
    grouped_df['rank_segment'] = grouped_df.apply(rank_segment, axis=1)
    
    return grouped_df

date = '2024-02-11'
num_months = 1  # Change this to 1, 2, or 6 as needed
rank_dataframe = rank_dataframe2(df, date, num_months)
print(len(rank_dataframe))
rank_dataframe.sort_values("average_sales",ascending=False).head(5)

1952


,foodcourt,restaurant,restaurantmenuitem,average_sales,rank,sales_segment,rank_segment
1325,63c1178d2381e7001bf38f10,63c3cd051a491b0017062b2b,649fbf3a061c49001bb67a08,517.666667,1,>100,1-100
1334,63c1178d2381e7001bf38f10,63c3cd051a491b0017062b2b,649fbf3a061c49001bb67a1d,447.750000,2,>100,1-100
1335,63c1178d2381e7001bf38f10,63c3cd051a491b0017062b2b,649fbf3a061c49001bb67a20,424.000000,3,>100,1-100
1330,63c1178d2381e7001bf38f10,63c3cd051a491b0017062b2b,649fbf3a061c49001bb67a18,423.000000,4,>100,1-100
1327,63c1178d2381e7001bf38f10,63c3cd051a491b0017062b2b,649fbf3a061c49001bb67a0f,422.000000,5,>100,1-100


In [6]:
def rank_pivot(df):
    # Pivot the DataFrame on 'rank_segment' and 'sales_segment' and take counts
    pivot_df = df.pivot_table(index='rank_segment', columns='sales_segment', values='rank', aggfunc='count', fill_value=0)
    
    # Reset the index of the pivoted DataFrame
    pivot_df = pivot_df.reset_index()
    
    # Reorder the columns as per the specified order
    pivot_df = pivot_df[["rank_segment", ">100", "75-100", "50-75", "25-50", "10-25", "<10"]]
    
    return pivot_df
pivot_summary = rank_pivot(rank_dataframe)
pivot_summary

sales_segment,rank_segment,>100,75-100,50-75,25-50,10-25,<10
0,1-100,77,23,0,0,0,0
1,100-200,0,29,71,0,0,0
2,200-300,0,0,6,94,0,0
3,300-400,0,0,0,100,0,0
4,400-500,0,0,0,17,83,0
5,>500,0,0,0,0,322,1130


In [7]:
def active_sku(df, train_date, num_days):
    # Convert date to datetime object
    end_date = datetime.strptime(train_date, '%Y-%m-%d')
    
    # Calculate start date based on end date and number of days
    start_date = end_date - timedelta(days=num_days) + timedelta(days=1)  
    
    # Filter dataframe based on date range
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    grouped_df = filtered_df.groupby(['foodcourt', 'restaurant', 'restaurantmenuitem']).agg({'total_count': 'sum'}).reset_index()
    active_data = grouped_df[grouped_df["total_count"] >=20 ]
    active_data['sku_concat'] = active_data['foodcourt'] + '-' + active_data['restaurant'] + '-' + active_data['restaurantmenuitem']
    return active_data

train_date = '2024-02-11'
num_days = 14
active_dataframe = active_sku(df, train_date, num_days)
#sku_list = df2["sku_concat"].unique()

In [8]:
#Filters the dataframe on the basis of the active sku
def filter_by_sku_list(df, active_dataframe):
    sku_list = active_dataframe["sku_concat"].unique()
    df['sku_concat'] = df['foodcourt'] + '-' + df['restaurant'] + '-' + df['restaurantmenuitem']
    filtered_data = df[df['sku_concat'].isin(sku_list)]
    filtered_data = filtered_data.drop(['sku_concat'],axis = 1)
    return filtered_data
base_data = filter_by_sku_list(df, active_dataframe)
len(base_data)

70090

In [9]:
min(base_data.date)

Timestamp('2023-09-01 00:00:00')

In [10]:
base_data.head()
rank_dataframe_b = rank_dataframe2(base_data, date, num_months)
pivot_summary_b = rank_pivot(rank_dataframe_b)
pivot_summary_b

sales_segment,rank_segment,>100,75-100,50-75,25-50,10-25,<10
0,1-100,55,39,6,0,0,0
1,100-200,0,0,59,41,0,0
2,200-300,0,0,0,100,0,0
3,300-400,0,0,0,22,78,0
4,400-500,0,0,0,0,100,0
5,>500,0,0,0,0,118,355


In [11]:
## Functions
# add missing date for each date range combination
def add_missing_dates(start_date, end_date, df):
    # Create a date range between start_date and end_date
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create a DataFrame from the date range
    date_df = pd.DataFrame({'date': date_range})
    
    # Merge item_data with date_df to ensure all data is present
    merged_df = pd.merge(date_df, df, on='date', how='left')
    merged_df['foodcourt'].fillna(method='ffill', inplace=True)
    merged_df['restaurant'].fillna(method='ffill', inplace=True)
    merged_df['restaurantmenuitem'].fillna(method='ffill', inplace=True)
    merged_df['foodcourt'].fillna(method='bfill', inplace=True)
    merged_df['restaurant'].fillna(method='bfill', inplace=True)
    merged_df['restaurantmenuitem'].fillna(method='bfill', inplace=True)
    merged_df['total_count'].fillna(0, inplace=True)
    
    # Fill missing values with 0s
    #merged_df.fillna(0, inplace=True)
    
    return merged_df

## MAPE function creation
data = {
    'actual': [10, 20, 29, 0, 0],
    'predicted': [12, 18, 28, 0, 88]
}

# Create DataFrame
data = pd.DataFrame(data)

def acc_calculation(df,actual='actual',predicted='predicted'):
    # Add 'error' column to DataFrame
    df['error'] = abs(df[actual] - df[predicted]) / df[actual]
    
    # Handle special cases where actual is zero
    df.loc[(df[actual] == 0) & (df[predicted] == 0), 'error'] = 0
    df.loc[(df[actual] == 0) & (df[predicted] != 0), 'error'] = 1
    df['error'] = df['error'].clip(upper=1)

    df['MAPE'] = df['error'].mean()    
    df['Accuracy'] = 1- df['MAPE'] 
    return df
data = acc_calculation(data)
data

#Model

### XGB Model

def run_xgboost(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Extract target variable
    y_train = train_data['total_count']
    y_test = test_data['total_count']
    
    # Train the XGBoost model
    model = xgb.XGBRegressor()
    X_train = train_data['date'].dt.dayofweek.values.reshape(-1, 1)  # Use day of week as feature
    model.fit(X_train, y_train)
    
    # Generate forecasts for the test data
    X_test = test_data['date'].dt.dayofweek.values.reshape(-1, 1)  # Use day of week as feature
    forecast = model.predict(X_test)
    forecast = np.floor(forecast)
    forecast[test_data['date'].dt.dayofweek.isin([5, 6])] = 0
    
    # Calculate R2 score
    r2 = r2_score(y_test, forecast)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test, forecast)
    accuracy_df = pd.DataFrame({
        'actual': y_test,
        'predicted': forecast
    })
    accuracy_df = acc_calculation(accuracy_df, actual='actual', predicted='predicted')
    accuracy = accuracy_df["Accuracy"]
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'date': test_data['date'],
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'actual': y_test,
        'forecast_xgb_norm': forecast,
        'r2': r2,
        'mape': mape,
        'model': "xgboost-norm",
        "accuracy_xgb_norm":accuracy
    })
    
    return output_df


def run_xgboost_lag(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create day_of_week feature
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Add lag_7 feature
    item_data['lag_7'] = item_data['total_count'].shift(7)
    #item_data  = item_data.fillna(0)
    item_data.reset_index(drop=True, inplace=True)
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Extract features and target variables
    X_train, y_train = train_data.drop(columns=['total_count',"foodcourt","restaurant","restaurantmenuitem","date"]), train_data['total_count']
    X_test, y_test = test_data.drop(columns=['total_count',"foodcourt","restaurant","restaurantmenuitem","date"]), test_data['total_count']

    # Train the XGBoost model
    model = xgb.XGBRegressor()  # Set enable_categorical to True
    model.fit(X_train, y_train)
    
    # Generate forecasts for the test data
    forecast = model.predict(X_test)
    forecast = np.floor(forecast)
    forecast[test_data['date'].dt.dayofweek.isin([5, 6])] = 0
    
    # Calculate R2 score
    r2 = r2_score(test_data['total_count'], forecast)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(test_data['total_count'], forecast)
    
    accuracy_df = pd.DataFrame({
        'actual': y_test,
        'predicted': forecast
    })
    accuracy_df = acc_calculation(accuracy_df, actual='actual', predicted='predicted')
    accuracy = accuracy_df["Accuracy"]
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'date': test_data['date'],
        'actual': y_test,
        'forecast_xgb_lag': forecast,
        'r2': r2,
        'mape': mape,
        "accuracy_xgb_lag":accuracy,
        'model': "xgboost-lag",
    })
    
    return output_df



def run_rf_lag(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create day_of_week feature
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Add lag_7 feature
    item_data['lag_7'] = item_data['total_count'].shift(7)
    item_data = item_data.fillna(0)
    item_data.reset_index(drop=True, inplace=True)
    
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Extract features and target variables
    X_train, y_train = train_data.drop(columns=['total_count', "foodcourt", "restaurant", "restaurantmenuitem", "date"]), train_data['total_count']
    X_test, y_test = test_data.drop(columns=['total_count', "foodcourt", "restaurant", "restaurantmenuitem", "date"]), test_data['total_count']

    # Train the RandomForest model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    # Generate forecasts for the test data
    forecast = model.predict(X_test)
    forecast = np.floor(forecast)
    forecast[test_data['date'].dt.dayofweek.isin([5, 6])] = 0
    
    # Calculate R2 score
    r2 = r2_score(test_data['total_count'], forecast)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(test_data['total_count'], forecast)
    
    accuracy_df = pd.DataFrame({
        'actual': y_test,
        'predicted': forecast
    })
    accuracy_df = acc_calculation(accuracy_df, actual='actual', predicted='predicted')
    accuracy = accuracy_df["Accuracy"]
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'date': test_data['date'],
        'actual': y_test,
        'forecast_rf_lag': forecast,
        'r2': r2,
        'mape': mape,
        "accuracy_rf_lag": accuracy,
        'model': "randomforest-lag",
    })
    
    return output_df

# LightGBM


def run_lgb_lag(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create day_of_week feature
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Add lag_7 feature
    item_data['lag_7'] = item_data['total_count'].shift(7)
    item_data = item_data.fillna(0)
    item_data.reset_index(drop=True, inplace=True)
    
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Extract features and target variables
    X_train, y_train = train_data.drop(columns=['total_count', "foodcourt", "restaurant", "restaurantmenuitem", "date"]), train_data['total_count']
    X_test, y_test = test_data.drop(columns=['total_count', "foodcourt", "restaurant", "restaurantmenuitem", "date"]), test_data['total_count']

    # Train the RandomForest model
    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)
    
    # Generate forecasts for the test data
    forecast = model.predict(X_test)
    forecast = np.floor(forecast)
    forecast[test_data['date'].dt.dayofweek.isin([5, 6])] = 0
    
    # Calculate R2 score
    r2 = r2_score(test_data['total_count'], forecast)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(test_data['total_count'], forecast)
    
    accuracy_df = pd.DataFrame({
        'actual': y_test,
        'predicted': forecast
    })
    accuracy_df = acc_calculation(accuracy_df, actual='actual', predicted='predicted')
    accuracy = accuracy_df["Accuracy"]
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'date': test_data['date'],
        'actual': y_test,
        'forecast_rf_lag': forecast,
        'r2': r2,
        'mape': mape,
        "accuracy_lgb_lag": accuracy,
        'model': "lgb_lag",
    })
    
    return output_df




from prophet import Prophet


def run_prophet_lag2(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create day_of_week feature
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Add lag_7 feature
    item_data['lag_7'] = item_data['total_count'].shift(7)
    #item_data  = item_data.fillna(0)
    item_data.reset_index(drop=True, inplace=True)
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Prepare DataFrame for Prophet
    prophet_train_data = train_data[['date', 'total_count']].rename(columns={'date': 'ds', 'total_count': 'y'})
    
    # Fit Prophet model
    model = Prophet()
    model.fit(prophet_train_data)
    
    # Make future DataFrame for Prophet
    future = model.make_future_dataframe(periods=len(test_data))
    
    # Add lag_7 feature
    #future['lag_7'] = item_data['total_count'].shift(7)[-len(test_data):].values
    
    # Predict with Prophet
    forecast = model.predict(future)
    
    # Extract forecasted values for the test period
    forecast_test = forecast.iloc[-len(test_data):]
    forecast_test['yhat'] = forecast_test['yhat'].apply(lambda x: max(0, int(np.floor(x))))
    forecast_test.loc[forecast_test['ds'].dt.dayofweek.isin([5, 6]), 'yhat'] = 0
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'date': test_data['date'],
        'forecast': forecast_test['yhat'],
        'r2': r2_score(test_data['total_count'], forecast_test['yhat']),
        #'mape': mean_absolute_percentage_error(test_data['total_count'], forecast_test['yhat']),
        'actual': test_data['total_count']
    })
    #output_df = acc_calculation(output_df, actual='total_count', predicted='forecast')
    output_df = acc_calculation(output_df, actual='actual', predicted='forecast')
    output_df = output_df.rename(columns={'Accuracy': 'accuracy_prophet'})
    return output_df
import tqdm as notebook_tqdm
import pandas as pd
import numpy as np
#from fbprophet import Prophet
from sklearn.metrics import r2_score

def run_prophet_lag3(item_data, train_date, test_date):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create day_of_week feature
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Add lag_7 feature
    item_data['lag_7'] = item_data['total_count'].shift(7)
    item_data['lag_7'] = item_data['lag_7'].fillna(0)  # Fill missing values with zeros
    item_data.reset_index(drop=True, inplace=True)
    
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Prepare DataFrame for Prophet
    prophet_train_data = train_data[['date', 'total_count', 'day_of_week', 'lag_7']].rename(columns={'date': 'ds', 'total_count': 'y'})
    
    # Fit Prophet model
    model = Prophet()
    model.add_regressor('day_of_week')
    model.add_regressor('lag_7')
    model.fit(prophet_train_data)
    
    # Make future DataFrame for Prophet
    future = model.make_future_dataframe(periods=len(test_data))
    future['day_of_week'] = future['ds'].dt.dayofweek
    future['lag_7'] = item_data['total_count'].shift(7)[-len(future):].fillna(0).values
    
    # Predict with Prophet
    forecast = model.predict(future)
    
    # Extract forecasted values for the test period
    forecast_test = forecast.iloc[-len(test_data):]
    forecast_test['yhat'] = forecast_test['yhat'].apply(lambda x: max(0, int(np.floor(x))))
    forecast_test.loc[forecast_test['ds'].dt.dayofweek.isin([5, 6]), 'yhat'] = 0
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'date': test_data['date'],
        'forecast': forecast_test['yhat'],
        'r2': r2_score(test_data['total_count'], forecast_test['yhat']),
        'actual': test_data['total_count']
    })
    #output_df = acc_calculation(output_df, actual='total_count', predicted='forecast')
    output_df = acc_calculation(output_df, actual='actual', predicted='forecast')
    output_df = output_df.rename(columns={'Accuracy': 'accuracy_prophet_multi'})
    return output_df

def time_series_forecasting_auto_arima(item_data, train_date, test_date, max_p=7, max_q=7):
    # Order the data based on date
    item_data = item_data.sort_values(by='date')
    
    # Create lag features
    item_data['lag_7'] = item_data['total_count'].shift(7)
    item_data['day_of_week'] = item_data['date'].dt.dayofweek
    
    # Drop rows with NaN values created by lag
    item_data = item_data.dropna()
    
    # Split the data into train and test based on train_date and test_date
    train_data = item_data[item_data['date'] <= train_date]
    test_data = item_data[(item_data['date'] > train_date) & (item_data['date'] <= test_date)]
    
    # Check if the test dataset is empty
    if test_data.empty:
        raise ValueError("Test dataset is empty. Please check your input data.")
    
    # Prepare the exogenous variables (lag_7 and day_of_week)
    exog_train = train_data[['lag_7', 'day_of_week']]
    exog_test = test_data[['lag_7', 'day_of_week']]
    
    # Fit auto ARIMA model
    arima_model = auto_arima(train_data['total_count'], seasonal=False, max_p=7, max_q=7, stepwise=False,
                             exogenous=exog_train, trace=True, error_action='ignore', suppress_warnings=True)
    
    # Forecast for the test period
    forecast, conf_int = arima_model.predict(n_periods=len(test_data), exogenous=exog_test, return_conf_int=True)
    
    # Set predictions for Saturday (5) and Sunday (6) to zero
    forecast = pd.Series(forecast, index=test_data.index)
    forecast[test_data['date'].dt.dayofweek.isin([5, 6])] = 0
    
    # Calculate R2 score
    #r2 = r2_score(test_data['total_count'], forecast)
    
    # Calculate MAPE
    #mape = mean_absolute_percentage_error(test_data['total_count'], forecast)
    
    # Create accuracy DataFrame and calculate accuracy
    accuracy_df = pd.DataFrame({
        'actual': test_data['total_count'],
        'predicted': forecast
    })
    accuracy_df = acc_calculation(accuracy_df, actual='actual', predicted='predicted')
    accuracy = accuracy_df["Accuracy"]
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        'date': test_data['date'],
        'foodcourt': test_data['foodcourt'],
        'restaurant': test_data['restaurant'],
        'restaurantmenuitem': test_data['restaurantmenuitem'],
        'actual': test_data['total_count'],
        'forecast_arima': forecast,
        #'r2': r2,
        #'mape': mape,
        'model': "auto-arima",
        "accuracy_arima": accuracy
    })
    
    return output_df




In [12]:
train_start= '2023-12-25'
train_end= '2024-02-04'
test_end= '2024-02-11'

In [13]:
# Creating the base data frame
data = []
forecast_xgb_norm = []
forecast_xgb_lag  = []
forecast_rf_lag  = []
forecast_lgb_lag  = []
forecast_prophet  = []
forecast_prophet_multi  = []
#forecast_arima  = []
sales_data = base_data.copy()
sales_data = sales_data[(sales_data["date"] >= train_start) & (sales_data["date"] <= test_end)]
# sales_data['weekday'] = sales_data['date'].dt.weekday
unique_foodcourts = sales_data['foodcourt'].unique()
for foodcourt in unique_foodcourts:
    # Subset data for the current food court
    foodcourt_data = sales_data[sales_data['foodcourt'] == foodcourt]
    
    # Unique restaurants within the current food court
    unique_restaurants_foodcourt = foodcourt_data['restaurant'].unique()
    
    # Middle loop for restaurants
    for restaurant in unique_restaurants_foodcourt:
        # Subset data for the current restaurant within the current food court
        restaurant_data = foodcourt_data[foodcourt_data['restaurant'] == restaurant]
        
        # Unique items for the current restaurant
        unique_items_restaurant = restaurant_data['restaurantmenuitem'].unique()
        
        # Inner loop for items within the current restaurant
        for item in unique_items_restaurant:
            # Subset data for the current item within the current restaurant and food court
            item_data = restaurant_data[restaurant_data['restaurantmenuitem'] == item]
            
            # Fill missing dates for the current item
            item_data2 = add_missing_dates(train_start, test_end, item_data)
            data.append(item_data2)
            
            #XGB with day of week
            forecast_output_xgb = run_xgboost(item_data2, train_end,test_end)
            forecast_xgb_norm.append(forecast_output_xgb)
                        
            #XGB with lag
            forecast_output_xgb_lag = run_xgboost_lag(item_data2, train_end,test_end)
            forecast_xgb_lag.append(forecast_output_xgb_lag)

            #RF with lag
            forecast_output_rf_lag = run_rf_lag(item_data2, train_end,test_end)
            forecast_rf_lag.append(forecast_output_rf_lag)

            #LGBM 
            # forecast_output_lgb_lag = run_lgb_lag(item_data2, train_end,test_end)
            # forecast_lgb_lag.append(forecast_output_lgb_lag)

            #Prophet
            forecast_output_prophet= run_prophet_lag2(item_data2, train_end,test_end)
            forecast_prophet.append(forecast_output_prophet)
            # forecast_prophet.append(forecast_output_prophet)

            forecast_output_prophet= run_prophet_lag3(item_data2, train_end,test_end)
            forecast_prophet_multi.append(forecast_output_prophet)
            # forecast_prophet.append(forecast_output_prophet)

            #forecast_output_arima= time_series_forecasting_auto_arima(item_data2, train_end,test_end)
            #forecast_arima.append(forecast_output_arima)


            #time_series_forecasting_auto_arima

# Concatenate all forecast results into a single DataFrame
data = pd.concat(data, ignore_index=True)
forecast_xgb_norm = pd.concat(forecast_xgb_norm, ignore_index=True)
forecast_xgb_lag = pd.concat(forecast_xgb_lag, ignore_index=True)
forecast_rf_lag = pd.concat(forecast_rf_lag, ignore_index=True)
#forecast_lgb_lag = pd.concat(forecast_lgb_lag, ignore_index=True)
forecast_prophet = pd.concat(forecast_prophet, ignore_index=True)
forecast_prophet_multi = pd.concat(forecast_prophet_multi, ignore_index=True)
#forecast_arima = pd.concat(forecast_arima, ignore_index=True)

09:24:10 - cmdstanpy - INFO - Chain [1] start processing
09:24:10 - cmdstanpy - INFO - Chain [1] done processing
09:24:10 - cmdstanpy - INFO - Chain [1] start processing
09:24:10 - cmdstanpy - INFO - Chain [1] done processing
09:24:10 - cmdstanpy - INFO - Chain [1] start processing
09:24:10 - cmdstanpy - INFO - Chain [1] done processing
09:24:11 - cmdstanpy - INFO - Chain [1] start processing
09:24:11 - cmdstanpy - INFO - Chain [1] done processing
09:24:11 - cmdstanpy - INFO - Chain [1] start processing
09:24:11 - cmdstanpy - INFO - Chain [1] done processing
09:24:11 - cmdstanpy - INFO - Chain [1] start processing
09:24:11 - cmdstanpy - INFO - Chain [1] done processing
09:24:11 - cmdstanpy - INFO - Chain [1] start processing
09:24:11 - cmdstanpy - INFO - Chain [1] done processing
09:24:11 - cmdstanpy - INFO - Chain [1] start processing
09:24:11 - cmdstanpy - INFO - Chain [1] done processing
09:24:12 - cmdstanpy - INFO - Chain [1] start processing
09:24:12 - cmdstanpy - INFO - Chain [1]

In [14]:
forecast_xgb_norm.to_csv("forecast_xgb_norm.csv")
forecast_xgb_lag.to_csv("forecast_xgb_lag.csv")
forecast_prophet.to_csv("forecast_prophet.csv")
forecast_prophet_multi.to_csv("forecast_prophet_multi.csv")
forecast_rf_lag.to_csv("forecast_rf_lag.csv")
#forecast_lgb_lag.to_csv("forecast_lgb_lag.csv")
#forecast_arima.to_csv("forecast_arima.csv")

In [15]:
forecast_prophet_multi

,foodcourt,restaurant,restaurantmenuitem,date,forecast,r2,actual,error,MAPE,accuracy_prophet_multi
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,2024-02-05,47,-0.201262,0.0,1.000000,0.857143,0.142857
1,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,2024-02-06,26,-0.201262,0.0,1.000000,0.857143,0.142857
2,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,2024-02-07,24,-0.201262,0.0,1.000000,0.857143,0.142857
3,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,2024-02-08,24,-0.201262,0.0,1.000000,0.857143,0.142857
4,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,2024-02-09,40,-0.201262,0.0,1.000000,0.857143,0.142857
...,...,...,...,...,...,...,...,...,...,...
6806,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b9,2024-02-07,7,0.394428,6.0,0.166667,0.168290,0.831710
6807,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b9,2024-02-08,5,0.394428,8.0,0.375000,0.168290,0.831710
6808,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b9,2024-02-09,4,0.394428,4.0,0.000000,0.168290,0.831710
6809,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b9,2024-02-10,0,0.394428,0.0,0.000000,0.168290,0.831710


In [16]:
# deep_ar_multi = pd.read_csv("DeepAR_multi.csv")
# deep_ar_multi = deep_ar_multi[["foodcourt","restaurant","restaurantmenuitem","Accuracy_DeepAR_Multi"]].reset_index(drop= True).drop_duplicates()
# deep_ar_multi.head()

In [17]:
multi_variate = pd.read_csv("results_rocv2.csv")
#  multi_variate = pd.read_csv("results_5_11.csv")
multi_variate = multi_variate[["foodcourtid","restaurant","menuitemid","accuracy_lr","accuracy_xgb_multi","accuracy_rf_lag"]].drop_duplicates()
multi_variate = multi_variate.rename(columns={"foodcourtid":"foodcourt","menuitemid":"restaurantmenuitem","accuracy_rf_lag":"accuracy_rf_multi"})
print(len(multi_variate))
multi_variate.head()

973


,foodcourt,restaurant,restaurantmenuitem,accuracy_lr,accuracy_xgb_multi,accuracy_rf_multi
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,0.003277,0.003604,0.142857
7,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c77,0.129870,0.000000,0.142857
14,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f17,0.030220,0.071429,0.142857
21,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f1e,0.028571,0.071429,0.142857
28,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f21,0.034529,0.065674,0.142857


In [18]:
deep_ar_uni = pd.read_csv("deepar_uni_5_11.csv")
deep_ar_uni = deep_ar_uni[["item_id","Accuracy"]].drop_duplicates()
deep_ar_uni = deep_ar_uni.rename(columns={"Accuracy":"accuracy_deepar_uni"})
deep_ar_multi = pd.read_csv("deepar_multi_5_11.csv")
deep_ar_multi = deep_ar_multi[["item_id","Accuracy"]].drop_duplicates()
deep_ar_multi = deep_ar_multi.rename(columns={"Accuracy":"accuracy_deepar_multi"})
deep_ar = pd.merge(deep_ar_uni, deep_ar_multi, on=['item_id'], how='left')
print(len(deep_ar))

973


In [19]:
deep_ar[['foodcourt', 'restaurant', 'restaurantmenuitem']] = deep_ar['item_id'].str.split('_', expand=True)
deep_ar = deep_ar[['foodcourt', 'restaurant', 'restaurantmenuitem','accuracy_deepar_uni','accuracy_deepar_multi']]
print(len(deep_ar))
deep_ar.head()

973


,foodcourt,restaurant,restaurantmenuitem,accuracy_deepar_uni,accuracy_deepar_multi
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,0.285714,0.285714
1,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c77,0.142857,0.142857
2,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f17,0.142857,0.142857
3,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f1e,0.142857,0.142857
4,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f21,0.142857,0.142857


In [20]:
xgb_norm = forecast_xgb_norm[["foodcourt","restaurant","restaurantmenuitem","accuracy_xgb_norm"]].reset_index(drop= True).drop_duplicates()
xgb_lag = forecast_xgb_lag[["foodcourt","restaurant","restaurantmenuitem","accuracy_xgb_lag"]].reset_index(drop= True).drop_duplicates()
rf_lag = forecast_rf_lag[["foodcourt","restaurant","restaurantmenuitem","accuracy_rf_lag"]].reset_index(drop= True).drop_duplicates()
#lgb_lag = forecast_lgb_lag[["foodcourt","restaurant","restaurantmenuitem","accuracy_lgb_lag"]].reset_index(drop= True).drop_duplicates()
prophet = forecast_prophet[["foodcourt","restaurant","restaurantmenuitem","accuracy_prophet"]].reset_index(drop= True).drop_duplicates()
prophet_multi = forecast_prophet_multi[["foodcourt","restaurant","restaurantmenuitem","accuracy_prophet_multi"]].reset_index(drop= True).drop_duplicates()
#arima = forecast_arima[["foodcourt","restaurant","restaurantmenuitem","accuracy_arima"]].reset_index(drop= True).drop_duplicates()
#xgb_multi = pd.read_csv("xgb_summary.csv")
##lr = pd.read_csv("lr_summary.csv")
# deep_ar = pd.read_csv("deep_ar_summary.csv")
# deep_ar_multi = pd.read_csv("deep_ar_summary.csv")
output = pd.merge(xgb_lag, xgb_norm, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output = pd.merge(output, rf_lag, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output = pd.merge(output, prophet, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
#output = pd.merge(output, xgb_multi, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
#output = pd.merge(output, lr, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output = pd.merge(output, prophet_multi, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output = pd.merge(output, deep_ar, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output = pd.merge(output, multi_variate, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
#output = pd.merge(output, deep_ar_multi, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output['max_accuracy'] = output[['accuracy_xgb_lag','accuracy_rf_lag', 'accuracy_xgb_norm','accuracy_prophet',"accuracy_prophet_multi","accuracy_deepar_multi","accuracy_deepar_uni","accuracy_lr","accuracy_xgb_multi","accuracy_rf_multi"]].max(axis=1)
output['best_model'] = output[['accuracy_xgb_lag', 'accuracy_rf_lag','accuracy_xgb_norm','accuracy_prophet',"accuracy_prophet_multi","accuracy_deepar_multi","accuracy_deepar_uni","accuracy_lr","accuracy_xgb_multi","accuracy_rf_multi"]].idxmax(axis=1)

In [21]:
output

,foodcourt,restaurant,restaurantmenuitem,accuracy_xgb_lag,accuracy_xgb_norm,accuracy_rf_lag,accuracy_prophet,accuracy_prophet_multi,accuracy_deepar_uni,accuracy_deepar_multi,accuracy_lr,accuracy_xgb_multi,accuracy_rf_multi,max_accuracy,best_model
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,0.571429,0.428571,0.428571,0.142857,0.142857,0.285714,0.285714,0.003277,0.003604,0.142857,0.571429,accuracy_xgb_lag
1,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c77,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.129870,0.000000,0.142857,0.285714,accuracy_xgb_lag
2,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f17,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.030220,0.071429,0.142857,0.285714,accuracy_xgb_lag
3,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f1e,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.028571,0.071429,0.142857,0.285714,accuracy_xgb_lag
4,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f21,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.034529,0.065674,0.142857,0.142857,accuracy_xgb_lag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,64ef112a15331c00185764b6,65990a9504979b001bbb5bab,6599193f649b99001be132d9,1.000000,1.000000,1.000000,0.285714,0.285714,0.285714,0.285714,0.000000,1.000000,0.285714,1.000000,accuracy_xgb_lag
969,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23ad,0.771845,0.791745,0.902527,0.833743,0.838746,0.888679,0.898406,0.454175,0.491881,0.799063,0.902527,accuracy_rf_lag
970,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b1,0.644120,0.793474,0.693401,0.701034,0.691240,0.629985,0.751892,0.181058,0.226275,0.677650,0.793474,accuracy_xgb_norm
971,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b5,0.714153,0.732135,0.747020,0.665634,0.747719,0.498501,0.495038,0.339527,0.280120,0.632501,0.747719,accuracy_prophet_multi


In [41]:
rocv2 = output[["foodcourt","restaurant","restaurantmenuitem","max_accuracy","best_model"]]
rocv2 = rocv2.rename(columns={'max_accuracy':'r2_accuracy','best_model':'r2_model'})
rocv2.to_excel('rocv2_out.xlsx')
rocv2

,foodcourt,restaurant,restaurantmenuitem,r2_accuracy,r2_model
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,0.571429,accuracy_xgb_lag
1,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c77,0.285714,accuracy_xgb_lag
2,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f17,0.285714,accuracy_xgb_lag
3,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f1e,0.285714,accuracy_xgb_lag
4,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f21,0.142857,accuracy_xgb_lag
...,...,...,...,...,...
968,64ef112a15331c00185764b6,65990a9504979b001bbb5bab,6599193f649b99001be132d9,1.000000,accuracy_xgb_lag
969,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23ad,0.902527,accuracy_rf_lag
970,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b1,0.793474,accuracy_xgb_norm
971,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b5,0.747719,accuracy_prophet_multi


In [23]:
def segment_max_accuracy(accuracy):
    if accuracy >= 0.90:
        return '>90'
    elif .80 <= accuracy < .90:
        return '80-90'
    elif .70 <= accuracy < .80:
        return '70-80'
    elif .50 <= accuracy < .70:
        return '50-70'
    else:
        return '<50'

def add_max_accuracy_segment(df):
    df['max_accuracy_segment'] = df['max_accuracy'].apply(segment_max_accuracy)
    return df

# Apply the function to your DataFrame
output = add_max_accuracy_segment(output)
rank_dataframe_segment =  rank_dataframe[["foodcourt","restaurant","restaurantmenuitem","sales_segment","rank_segment"]]
output = pd.merge(output, rank_dataframe_segment, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output.to_csv("output_summary.csv")

In [24]:
pivot_table_df = output.pivot_table(index='max_accuracy_segment',  aggfunc='size', fill_value=0)
desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex(index=desired_order_rows)
pivot_table_df

max_accuracy_segment
>90      182
80-90    376
70-80    207
50-70    165
<50       43
dtype: int64

In [25]:
pivot_table_df = output.pivot_table(index='sales_segment', columns='max_accuracy_segment', aggfunc='size', fill_value=0)
desired_order_rows = ['>100', '75-100','50-75','25-50',  '10-25',  '<10' ]
desired_order_columns = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex(index=desired_order_rows, columns=desired_order_columns)

pivot_table_df
#pivot_table_df["sales_segment","max_accuracy_segment",">100","75-100","50-75","25-50","10-25",">100"]

max_accuracy_segment,>90,80-90,70-80,50-70,<50
sales_segment,,,,,
>100,25,21,5,2,2
75-100,13,23,2,1,0
50-75,26,27,5,4,3
25-50,45,75,21,16,6
10-25,51,130,60,40,15
<10,22,100,114,102,17


In [26]:
pivot_table_df = output.pivot_table(index='rank_segment', columns='max_accuracy_segment', aggfunc='size', fill_value=0)
desired_order_rows = ['1-100', '100-200','200-300','300-400',  '400-500',  '>500' ]
desired_order_columns = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex(index=desired_order_rows, columns=desired_order_columns)
pivot_table_df

max_accuracy_segment,>90,80-90,70-80,50-70,<50
rank_segment,,,,,
1-100,29,34,5,2,2
100-200,31,37,5,5,3
200-300,25,27,10,6,4
300-400,22,40,11,7,2
400-500,13,39,11,13,6
>500,62,199,165,132,26


In [27]:
base_data2 = base_data[(base_data['date']>=train_start) & (base_data['date']<=train_end) ]
base_data2 = base_data2.groupby(['foodcourt','restaurant','restaurantmenuitem']).agg({'total_count': 'sum'}).reset_index()
output = pd.merge(output, base_data2, on=['foodcourt', 'restaurant', 'restaurantmenuitem'], how='left')
output.to_csv("output_with_sales.csv")

In [28]:
pivot_table_df = output.pivot_table(index='max_accuracy_segment', values=['total_count', 'foodcourt'], aggfunc={'total_count': 'sum', 'foodcourt': 'count'})
desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex(index=desired_order_rows)
pivot_table_df

,foodcourt,total_count
max_accuracy_segment,,
>90,182,109787.0
80-90,376,164490.0
70-80,207,67048.0
50-70,165,33698.0
<50,43,10981.0


In [29]:
output

,foodcourt,restaurant,restaurantmenuitem,accuracy_xgb_lag,accuracy_xgb_norm,accuracy_rf_lag,accuracy_prophet,accuracy_prophet_multi,accuracy_deepar_uni,accuracy_deepar_multi,accuracy_lr,accuracy_xgb_multi,accuracy_rf_multi,max_accuracy,best_model,max_accuracy_segment,sales_segment,rank_segment,total_count
0,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c5b,0.571429,0.428571,0.428571,0.142857,0.142857,0.285714,0.285714,0.003277,0.003604,0.142857,0.571429,accuracy_xgb_lag,50-70,>100,1-100,563.0
1,5b964b50a6fb5422abb2fda7,5b964a709153de1766fcf94c,65774ecf4fb51d001be77c77,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.129870,0.000000,0.142857,0.285714,accuracy_xgb_lag,<50,50-75,100-200,1209.0
2,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f17,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.030220,0.071429,0.142857,0.285714,accuracy_xgb_lag,<50,50-75,100-200,739.0
3,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f1e,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.028571,0.071429,0.142857,0.285714,accuracy_xgb_lag,<50,10-25,400-500,285.0
4,5b964b50a6fb5422abb2fda7,5c406ac021301e2fa8093d09,646b51e03d50300018af7f21,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.034529,0.065674,0.142857,0.142857,accuracy_xgb_lag,<50,>100,1-100,2655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,64ef112a15331c00185764b6,65990a9504979b001bbb5bab,6599193f649b99001be132d9,1.000000,1.000000,1.000000,0.285714,0.285714,0.285714,0.285714,0.000000,1.000000,0.285714,1.000000,accuracy_xgb_lag,>90,<10,>500,20.0
969,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23ad,0.771845,0.791745,0.902527,0.833743,0.838746,0.888679,0.898406,0.454175,0.491881,0.799063,0.902527,accuracy_rf_lag,>90,25-50,200-300,967.0
970,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b1,0.644120,0.793474,0.693401,0.701034,0.691240,0.629985,0.751892,0.181058,0.226275,0.677650,0.793474,accuracy_xgb_norm,70-80,25-50,200-300,885.0
971,650eaa5d1b1432001b409943,650ead650b6ef7001bcbdca6,6517e4e43a3ee4001bbe23b5,0.714153,0.732135,0.747020,0.665634,0.747719,0.498501,0.495038,0.339527,0.280120,0.632501,0.747719,accuracy_prophet_multi,70-80,<10,>500,211.0


In [30]:
pivot_table_df = output.pivot_table(index='best_model', values=['total_count', 'foodcourt'],columns='max_accuracy_segment', aggfunc={'total_count': 'sum', 'foodcourt': 'count'})
desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex( )
pivot_table_df

foodcourt                         total_count           \
max_accuracy_segment       50-70 70-80 80-90   <50   >90       50-70    70-80   
best_model                                                                      
accuracy_deepar_multi       14.0  19.0  41.0   5.0  10.0      3714.0  10518.0   
accuracy_deepar_uni         19.0  23.0  26.0   3.0   9.0      3115.0   4062.0   
accuracy_lr                  6.0   3.0   4.0   1.0   2.0      2136.0    433.0   
accuracy_prophet            23.0  35.0  76.0   1.0  23.0      3825.0   6717.0   
accuracy_prophet_multi      13.0  15.0  30.0   NaN  19.0      2983.0   6644.0   
accuracy_rf_lag             16.0  21.0  29.0   3.0  16.0      3846.0   8414.0   
accuracy_rf_multi           31.0  40.0  64.0   6.0  20.0      5640.0  21801.0   
accuracy_xgb_lag            26.0  30.0  64.0  23.0  49.0      4169.0   4869.0   
accuracy_xgb_multi           6.0   9.0  20.0   NaN  28.0      2787.0   1568.0   
accuracy_xgb_norm           11.0  12.0  22.0   1.0   6.0      1483.0   2022.0   

                                                  
max_accuracy_segment      80-90     <50      >90  
best_model                                        
accuracy_deepar_multi   31777.0  1535.0   7458.0  
accuracy_deepar_uni      6619.0   150.0   2971.0  
accuracy_lr              1030.0    89.0    201.0  
accuracy_prophet        44962.0    89.0  21322.0  
accuracy_prophet_multi  12863.0     NaN  21599.0  
accuracy_rf_lag         12897.0   305.0  13333.0  
accuracy_rf_multi       33648.0   396.0  15021.0  
accuracy_xgb_lag         9650.0  8361.0  18578.0  
accuracy_xgb_multi       3454.0     NaN   8185.0  
accuracy_xgb_norm        7590.0    56.0   1119.0

In [31]:
pivot_table_df = output.pivot_table(index='best_model', values=['foodcourt'],aggfunc={ 'foodcourt': 'count'})
#desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex( )
pivot_table_df

,foodcourt
best_model,
accuracy_deepar_multi,89
accuracy_deepar_uni,80
accuracy_lr,16
accuracy_prophet,158
accuracy_prophet_multi,77
accuracy_rf_lag,85
accuracy_rf_multi,161
accuracy_xgb_lag,192
accuracy_xgb_multi,63


In [32]:
pivot_table_df = output.pivot_table(index='best_model', values=[ 'foodcourt'],columns='max_accuracy_segment', aggfunc={ 'foodcourt': 'count'})
# desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex()
pivot_table_df

foodcourt                        
max_accuracy_segment       50-70 70-80 80-90   <50   >90
best_model                                              
accuracy_deepar_multi       14.0  19.0  41.0   5.0  10.0
accuracy_deepar_uni         19.0  23.0  26.0   3.0   9.0
accuracy_lr                  6.0   3.0   4.0   1.0   2.0
accuracy_prophet            23.0  35.0  76.0   1.0  23.0
accuracy_prophet_multi      13.0  15.0  30.0   NaN  19.0
accuracy_rf_lag             16.0  21.0  29.0   3.0  16.0
accuracy_rf_multi           31.0  40.0  64.0   6.0  20.0
accuracy_xgb_lag            26.0  30.0  64.0  23.0  49.0
accuracy_xgb_multi           6.0   9.0  20.0   NaN  28.0
accuracy_xgb_norm           11.0  12.0  22.0   1.0   6.0

In [33]:
meals= pd.read_excel("meals_std.xlsx")

In [34]:
meals = meals[["restaurantmenuitem","menuitemname"]].drop_duplicates(subset=['restaurantmenuitem'], keep='first')
meals

,restaurantmenuitem,menuitemname
0,648f1dbbe42a8f00174bfa3b,Veg Dinner Meal
1,646e15e20ecce00017753b9c,Aloo Paratha With Pickle & Butter
2,641c2ecbe3642c0017428534,Chicken Tikka Biryani
3,641e96130360260017dc33b1,Mini Chicken Thali
4,65644d7aa1cdf9001b15d6bd,Non Veg Combo
...,...,...
111883,64c89865eb8572001b22d414,Plain Biryani
111897,64f0422fdd06d3001ba97736,Plain Paratha With Paneer Preparation And salad
111921,64ca86f16fbdb000114fba13,Banjara Chicken Combo
111966,6513aaef06ef8d001bd1b40f,Thalipith with Curd


In [35]:
output = pd.merge(output, meals, on=[ 'restaurantmenuitem'], how='left')

In [36]:
len(output)

973

In [37]:
output.to_csv("output_with_itemname.csv")